In [1]:
# Import the required libraries and dependencies
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi


%matplotlib inline

In [2]:
# Load the environment variables from the .env file by calling the load_dotenv function
load_dotenv()

True

In [3]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2")

In [4]:
# Set start and end dates of 10 years back from your current date

tickers = ["BTC","ETH"]
timeframe = "1D"
start_date = pd.Timestamp("2011-04-27", tz="America/New_York").isoformat
end_date =  pd.Timestamp("2021-04-27", tz="America/New_York").isoformat



In [5]:
# The start and end dates should be updated with the information set above
ten_years_prices_crypto_df = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

ten_years_prices_crypto_df.dropna()

BTC                                       ETH  \
                              open     high      low    close  volume   open   
time                                                                           
2021-04-16 00:00:00-04:00  100.890  100.890  98.7700  99.2700  2795.0  28.49   
2021-04-19 00:00:00-04:00   99.120   99.130  98.9850  99.1012  3761.0  29.18   
2021-04-20 00:00:00-04:00   98.790   99.300  98.7900  98.8800  3259.0  28.95   
2021-04-21 00:00:00-04:00   99.010   99.010  98.9000  98.9000   386.0  28.24   
2021-04-22 00:00:00-04:00   98.925   99.040  98.8600  98.9100  1153.0  28.77   
2021-04-23 00:00:00-04:00   99.010   99.010  98.8801  98.9910  1425.0  28.83   
2021-04-26 00:00:00-04:00   99.000   99.000  98.9082  98.9100  1942.0  28.91   
2021-04-27 00:00:00-04:00   98.950   98.950  98.8200  98.8565  1022.0  28.76   
2021-04-28 00:00:00-04:00   98.880   98.880  98.7600  98.8300   543.0  29.68   
2021-04-29 00:00:00-04:00   98.810   98.900  98.8000  98.9000  1499.0  31.52   
2021-04-30 00:00:00-04:00   98.920   98.930  98.8800  98.9200  1681.0  30.66   
2021-05-03 00:00:00-04:00   98.910   98.945  98.9100  98.9450   782.0  29.41   
2021-05-04 00:00:00-04:00   98.960   99.050  98.9600  99.0500  2182.0  30.35   
2021-05-05 00:00:00-04:00   99.000   99.000  98.9940  99.0000  2573.0  31.14   
2021-05-06 00:00:00-04:00   99.100   99.101  99.0310  99.0750  3703.0  30.84   
2021-05-07 00:00:00-04:00   99.150   99.215  99.1500  99.1700  1315.0  31.57   
2021-05-10 00:00:00-04:00   99.200   99.200  99.0700  99.1200  2166.0  31.29   

                                                           
                            high      low   close  volume  
time                                                       
2021-04-16 00:00:00-04:00  29.43  28.4900  29.170  476979  
2021-04-19 00:00:00-04:00  29.40  28.6200  28.850  266218  
2021-04-20 00:00:00-04:00  29.17  28.0200  28.280  267794  
2021-04-21 00:00:00-04:00  28.95  28.2300  28.750  318757  
2021-04-22 00:00:00-04:00  29.06  28.5300  28.600  322321  
2021-04-23 00:00:00-04:00  29.24  28.5400  28.810  332659  
2021-04-26 00:00:00-04:00  29.74  28.4300  28.770  372335  
2021-04-27 00:00:00-04:00  29.12  28.6700  28.930  319516  
2021-04-28 00:00:00-04:00  31.62  29.3900  31.490  892692  
2021-04-29 00:00:00-04:00  31.90  30.7000  30.960  337363  
2021-04-30 00:00:00-04:00  30.70  28.0000  28.710  694084  
2021-05-03 00:00:00-04:00  30.77  29.3000  30.610  558507  
2021-05-04 00:00:00-04:00  31.15  29.6800  30.980  489529  
2021-05-05 00:00:00-04:00  31.49  30.2300  30.820  324149  
2021-05-06 00:00:00-04:00  31.12  30.1000  31.100  244711  
2021-05-07 00:00:00-04:00  32.10  30.9750  31.983  779865  
2021-05-10 00:00:00-04:00  32.15  31.0516  31.390  618579